In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

**Plan of action:**<br>
- Figure out which columns are needed in the stats data: pos,player,team,pass_yds,pass_td,pass_int,rec_yds,rec_tds,rush_yds,rush_td,surface,game_date
- Create a list of the injured players and make new df to merge that with the stats.
- Lump all three seasons to get averages or break it up for each season?
- Merge the offense/defense/kicking df's to line up the player's

### Pull in the data (injury reports/player stats)

In [ ]:
acl_report = pd.read_csv('../data/acl_report.csv', header = 1)

In [ ]:
concussion_report = pd.read_csv('../data/concussion_report.csv', header = 1)

In [ ]:
shoulder_report = pd.read_csv('../data/shoulder_report.csv', header = 1)

In [ ]:
ankle_report = pd.read_csv('../data/ankle_report.csv', header = 1)

In [ ]:
offense_stats = pd.read_csv('../data/nfl_offense.csv')

In [ ]:
defense_stats = pd.read_csv('../data/nfl_defense.csv')

In [ ]:
kicking_stats = pd.read_csv('../data/nfl_kicking.csv')

### Data Cleaning

In [ ]:
acl_report = acl_report[['Date', 'Team', 'Relinquished', 'Notes']]
concussion_report = concussion_report[['Date', 'Team', 'Relinquished', 'Notes']]
shoulder_report = shoulder_report[['Date', 'Team', 'Relinquished', 'Notes']]
ankle_report = ankle_report[['Date', 'Team', 'Relinquished', 'Notes']]

In [ ]:
acl_report = acl_report.rename(columns = {'Relinquished':'Player'})
concussion_report = concussion_report.rename(columns = {'Relinquished':'Player'})
shoulder_report = shoulder_report.rename(columns = {'Relinquished':'Player'})
ankle_report = ankle_report.rename(columns = {'Relinquished':'Player'})

In [ ]:
acl_report['Player'] = acl_report['Player'].str[2:]
concussion_report['Player'] = concussion_report['Player'].str[2:]
shoulder_report['Player'] = shoulder_report['Player'].str[2:]
ankle_report['Player'] = ankle_report['Player'].str[2:]

In [ ]:
acl_report

In [ ]:
offense_stats